In [3]:
import pandas as pd
from src.db_utils import load_model_wide

# ───────────────────────────────
def show_model_wide_columns() -> None:
    """
    Print all available columns from v_series_wide (excluding 'date').
    """
    wide = load_model_wide()
    cols = [c for c in wide.columns if c != "date"]
    print(f"\n🧹 Available columns ({len(cols)} fields):\n")
    for c in cols:
        print(f"- {c}")

# ───────────────────────────────
def load_fields(fields: str | list[str]) -> pd.DataFrame:
    """
    Return full history of one or multiple fields from v_series_wide.

    Parameters
    ----------
    fields : str or list of str
        Single field name or list of field names.

    Returns
    -------
    pd.DataFrame with date + selected fields
    """
    if isinstance(fields, str):
        fields = [fields]

    wide = load_model_wide()
    fields = ["date"] + fields  # always include date

    missing = [f for f in fields if f not in wide.columns]
    if missing:
        raise KeyError(f"Missing fields in v_series_wide: {missing}")

    return wide[fields].sort_values("date").reset_index(drop=True)


In [4]:
show_model_wide_columns()



🧹 Available columns (165 fields):

- baker_canada_montney_gas
- baker_canada_total_rigs
- baker_us_anadarko_gas
- baker_us_anadarko_oil
- baker_us_dj_niobrara_gas
- baker_us_dj_niobrara_oil
- baker_us_eagle_ford_gas
- baker_us_eagle_ford_oil
- baker_us_haynesville_gas
- baker_us_marcellus_gas
- baker_us_permian_gas
- baker_us_permian_oil
- baker_us_total_rigs
- baker_us_utica_gas
- baker_us_williston_gas
- baker_us_williston_oil
- balance_stock_change_misc_1qcy
- balance_stock_change_misc_1qny
- balance_stock_change_misc_2qcy
- balance_stock_change_misc_2qny
- balance_stock_change_misc_3qcy
- balance_stock_change_misc_3qny
- balance_stock_change_misc_4qcy
- balance_stock_change_misc_4qny
- brent_spot_price
- china_oil_demand_1qcy
- china_oil_demand_1qny
- china_oil_demand_2qcy
- china_oil_demand_2qny
- china_oil_demand_3qcy
- china_oil_demand_3qny
- china_oil_demand_4qcy
- china_oil_demand_4qny
- crude_runs_to_refineries
- cushing_crude_stocks
- days_supply_crude
- days_supply_distill

In [7]:
df = load_fields("field_production_of_crude")
print(df)
df['field_production_of_crude'].describe()

           date  field_production_of_crude
0    1982-08-20                        NaN
1    1982-08-27                        NaN
2    1982-09-24                        NaN
3    1982-10-01                        NaN
4    1982-10-08                        NaN
...         ...                        ...
6580 2025-04-16                        NaN
6581 2025-04-17                        NaN
6582 2025-04-18                    13460.0
6583 2025-04-21                        NaN
6584 2025-04-25                        NaN

[6585 rows x 2 columns]


count     2204.000000
mean      7777.524955
std       2336.947495
min       3813.000000
25%       5859.750000
50%       7150.500000
75%       8928.000000
max      13631.000000
Name: field_production_of_crude, dtype: float64

In [8]:
df2 = load_fields(["baker_us_total_rigs", "field_production_of_crude", "net_imports_of_crude"])
print(df2)


           date baker_us_total_rigs  field_production_of_crude  \
0    1982-08-20                None                        NaN   
1    1982-08-27                None                        NaN   
2    1982-09-24                None                        NaN   
3    1982-10-01                None                        NaN   
4    1982-10-08                None                        NaN   
...         ...                 ...                        ...   
6580 2025-04-16                None                        NaN   
6581 2025-04-17                None                        NaN   
6582 2025-04-18                None                    13460.0   
6583 2025-04-21                None                        NaN   
6584 2025-04-25                None                        NaN   

      net_imports_of_crude  
0                      NaN  
1                      NaN  
2                      NaN  
3                      NaN  
4                      NaN  
...                    ...  
6580

US Weekly Crude Oil Barrel Balance:

SUPPLYt =  Productiont           (EIA WCRFPUS2.W)
         + Net Importst          (WCRNTUS2.W)
         + SPR Flowst            (positive = draw)

DEMANDt =  Refinery Runst        (WCRRIUS2.W)
         + Exportst             (W_EPC0_EX0_NUS-Z00_MBBLD.W)

ΔInventoryt = SUPPLYt – DEMANDt            → should match
              Stocks_t – Stocks_{t-1}      (WCESTUS1.W)

(If they don’t, the plug is **“Adjustment / Unaccounted-for crude”**)
